<a href="https://colab.research.google.com/github/NamanNYC/Final_Project/blob/Steven/SQLRDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Database Setup
- Create RDS database tie-in with RDS
- Import Raw CSV tables into Datatbase
---

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-09 03:44:14--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2021-09-09 03:44:14 (11.2 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LMPT-Forest-Fires").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

Connection String to write into DB

In [ ]:
connection_string = 'lmpt-finalproject.coke2w4vs8wf.us-east-2.rds.amazonaws.com'
password = 'lmptdatagroup' #obtain this later from an add
database_name = 'postgres'

# Configure settings for RDS
mode = "append"
jdbc_url=f"jdbc:postgresql://{connection_string}:5432/{database_name}"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}


Import first CSV and write to Table



In [ ]:

from pyspark import SparkFiles
url = "https://wildfire.alberta.ca/resources/historical-data/documents/fires_2006to2018.csv"
spark.sparkContext.addFile(url)
fire_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)


In [ ]:
# Write review_id_df to table in RDS
df.write.jdbc(url=jdbc_url, table='fire_df', mode=mode, properties=config)

In [ ]:
fire_df.show()

+-----------+---------+---------+-------------+-------------------+-------------------+------------+----------+----------------------+-----------------------+------------------+-------------------+------------------------+----------------------+-------------------+-------------------+------------------+-------------------+--------------+---------+-------------------+-------------------+-------------------+------------------------+------------------------+-----------------+---------+----------------------+----------------------------+---------+------------------+-------------------+-----------+-------------------+-----------+----------+-----------+-------------------+-----------+
|fire_number|fire_name|fire_year|calendar_year|assessment_datetime|assessment_hectares|current_size|size_class|fire_location_latitude|fire_location_longitude|       fire_origin| general_cause_desc|industry_identifier_desc|responsible_group_desc|     activity_class|         true_cause|permit_detail_desc|    fire

Start cleaning Dataframes

In [ ]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import datetime as dt
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Clean data by creating DataFrame with selected columns
clean_fire_df = []
clean_fire_df = fire_df[['start_for_fire_date', 'discovered_date', 'fire_fighting_start_size','bh_hectares' ]]
clean_fire_df.show()

+-------------------+-------------------+------------------------+-----------+
|start_for_fire_date|    discovered_date|fire_fighting_start_size|bh_hectares|
+-------------------+-------------------+------------------------+-----------+
|2006-04-02 14:45:00|2006-04-02 14:27:00|                      NA|        0.2|
|2006-04-03 15:50:00|                 NA|                      NA|       0.01|
|2006-04-08 19:30:00|                 NA|                    0.01|       0.01|
|2006-04-13 17:52:00|2006-04-13 17:33:00|                      NA|       0.75|
|2006-04-14 15:31:00|2006-04-14 15:17:00|                      NA|       0.01|
|2006-04-17 15:10:00|2006-04-17 14:52:00|                      NA|       0.02|
|2006-04-18 17:01:00|2006-04-18 17:00:00|                      NA|        0.2|
|2006-04-19 20:40:00|2006-04-19 20:38:00|                      NA|        0.3|
|2006-04-20 12:07:00|2006-04-20 12:04:00|                      NA|        4.0|
|2006-04-22 19:52:00|2006-04-22 19:24:00|           

In [ ]:
# Review data types
clean_fire_df.dtypes

NameError: ignored

In [ ]:
# Convert objects to dates for the 2 columns
test[['start_for_fire_date', 'discovered_date']] = clean_fire_df[['start_for_fire_date', 'discovered_date']].apply(pd.to_datetime)
test1=clean_fire_df[['start_for_fire_date']] = pd.to_datetime(clean_fire_df[['start_for_fire_date']])
test2 = clean_fire_df[['start_for_fire_date']].astype('datetime64[ns]')

AttributeError: ignored

TypeError: ignored

In [ ]:

# Check that the changed worked
clean_fire_df.dtypes

[('start_for_fire_date', 'string'),
 ('discovered_date', 'string'),
 ('fire_fighting_start_size', 'string'),
 ('bh_hectares', 'double')]